In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
generator = 0

In [3]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\selfie2anime\trainB"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            if(np.array(x).shape == (128,128,3)):
                data.append(np.array(x))

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),128,128,3)
        return x_train

In [4]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 200

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
                generator = self.generator

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0] * 256)
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\painting_%d.png" % epoch)
        plt.close()

In [5]:
dcgan = DCGAN()
dcgan.train(epochs=10001, batch_size=40, save_interval=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 33, 33, 64)       

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.164251, acc.: 43.75%] [G loss: 0.135067]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.295489, acc.: 87.50%] [G loss: 3.127579]
2 [D loss: 0.667693, acc.: 63.75%] [G loss: 1.108415]
3 [D loss: 0.491656, acc.: 76.25%] [G loss: 2.358279]
4 [D loss: 0.194766, acc.: 95.00%] [G loss: 2.693780]
5 [D loss: 0.288209, acc.: 91.25%] [G loss: 2.415265]
6 [D loss: 0.114520, acc.: 100.00%] [G loss: 2.557825]
7 [D loss: 0.068619, acc.: 100.00%] [G loss: 1.920655]
8 [D loss: 0.056202, acc.: 100.00%] [G loss: 0.989274]
9 [D loss: 0.060230, acc.: 100.00%] [G loss: 0.471712]
10 [D loss: 0.064969, acc.: 100.00%] [G loss: 0.528686]
11 [D loss: 0.109674, acc.: 98.75%] [G loss: 0.473641]
12 [D loss: 0.150895, acc.: 98.75%] [G loss: 0.540266]
13 [D loss: 0.311383, acc.: 91.25%] [G loss: 1.122146]
14 [D loss: 0.579899, acc.: 70.00%] [G loss: 0.545761]
15 [D loss: 0.512288, acc.: 77.50%] [G loss: 1.580749]
16 [D loss: 1.950652, acc.: 16.25%] [G loss: 2.549710]
17 [D loss: 0.972744, acc.: 62.50%] [G loss: 2.098078]
18 [D loss: 0.613543, acc.: 71.25%] [G loss: 3.884120]
19 [D loss: 0.

...

9383 [D loss: 0.267755, acc.: 87.50%] [G loss: 5.690731]
9384 [D loss: 0.141036, acc.: 95.00%] [G loss: 7.066258]
9385 [D loss: 0.150981, acc.: 91.25%] [G loss: 5.067767]
9386 [D loss: 0.145770, acc.: 96.25%] [G loss: 5.943789]
9387 [D loss: 0.153427, acc.: 96.25%] [G loss: 5.113780]
9388 [D loss: 0.076986, acc.: 98.75%] [G loss: 5.420403]
9389 [D loss: 0.174315, acc.: 93.75%] [G loss: 6.169854]
9390 [D loss: 0.420857, acc.: 77.50%] [G loss: 5.957792]
9391 [D loss: 0.284175, acc.: 87.50%] [G loss: 5.928291]
9392 [D loss: 0.222301, acc.: 92.50%] [G loss: 7.268875]
9393 [D loss: 0.257335, acc.: 86.25%] [G loss: 6.989700]
9394 [D loss: 0.149136, acc.: 95.00%] [G loss: 3.794605]
9395 [D loss: 0.470142, acc.: 77.50%] [G loss: 10.647967]
9396 [D loss: 2.188613, acc.: 32.50%] [G loss: 9.002032]
9397 [D loss: 0.546414, acc.: 75.00%] [G loss: 5.445776]
9398 [D loss: 0.461604, acc.: 78.75%] [G loss: 5.540933]
9399 [D loss: 0.308701, acc.: 87.50%] [G loss: 5.068747]
9400 [D loss: 0.218125, acc.: 

9527 [D loss: 0.676980, acc.: 76.25%] [G loss: 4.742508]
9528 [D loss: 0.042272, acc.: 100.00%] [G loss: 6.031073]
9529 [D loss: 0.277453, acc.: 88.75%] [G loss: 5.797663]
9530 [D loss: 0.265439, acc.: 88.75%] [G loss: 7.344897]
9531 [D loss: 0.856718, acc.: 66.25%] [G loss: 5.339173]
9532 [D loss: 0.068534, acc.: 100.00%] [G loss: 6.809596]
9533 [D loss: 0.209090, acc.: 90.00%] [G loss: 6.207315]
9534 [D loss: 0.212263, acc.: 91.25%] [G loss: 4.947077]
9535 [D loss: 0.232378, acc.: 92.50%] [G loss: 5.874228]
9536 [D loss: 0.176295, acc.: 92.50%] [G loss: 5.617610]
9537 [D loss: 0.241357, acc.: 88.75%] [G loss: 6.668886]
9538 [D loss: 0.164320, acc.: 93.75%] [G loss: 5.398807]
9539 [D loss: 0.148255, acc.: 96.25%] [G loss: 5.086352]
9540 [D loss: 0.076331, acc.: 97.50%] [G loss: 5.576337]
9541 [D loss: 0.120881, acc.: 95.00%] [G loss: 4.895939]
9542 [D loss: 0.329770, acc.: 83.75%] [G loss: 6.702579]
9543 [D loss: 0.217382, acc.: 93.75%] [G loss: 4.772250]
9544 [D loss: 0.108995, acc.:

9671 [D loss: 0.341946, acc.: 87.50%] [G loss: 6.837214]
9672 [D loss: 0.641371, acc.: 72.50%] [G loss: 3.238952]
9673 [D loss: 0.063523, acc.: 98.75%] [G loss: 5.616735]
9674 [D loss: 0.302760, acc.: 85.00%] [G loss: 6.218423]
9675 [D loss: 0.261691, acc.: 83.75%] [G loss: 5.476658]
9676 [D loss: 0.368322, acc.: 85.00%] [G loss: 6.754732]
9677 [D loss: 0.434419, acc.: 81.25%] [G loss: 3.875360]
9678 [D loss: 0.238505, acc.: 88.75%] [G loss: 7.652157]
9679 [D loss: 0.874040, acc.: 65.00%] [G loss: 6.051491]
9680 [D loss: 0.150543, acc.: 92.50%] [G loss: 5.204387]
9681 [D loss: 0.157899, acc.: 96.25%] [G loss: 4.208019]
9682 [D loss: 0.292408, acc.: 87.50%] [G loss: 6.380800]
9683 [D loss: 0.462826, acc.: 83.75%] [G loss: 4.598177]
9684 [D loss: 0.290422, acc.: 85.00%] [G loss: 7.354715]
9685 [D loss: 0.475536, acc.: 80.00%] [G loss: 5.344074]
9686 [D loss: 0.161010, acc.: 96.25%] [G loss: 6.148141]
9687 [D loss: 0.136101, acc.: 96.25%] [G loss: 6.489057]
9688 [D loss: 0.245535, acc.: 8

9815 [D loss: 0.162028, acc.: 96.25%] [G loss: 5.260297]
9816 [D loss: 0.074953, acc.: 98.75%] [G loss: 4.992684]
9817 [D loss: 0.268308, acc.: 88.75%] [G loss: 7.718459]
9818 [D loss: 0.112842, acc.: 93.75%] [G loss: 6.880533]
9819 [D loss: 0.336459, acc.: 85.00%] [G loss: 4.596004]
9820 [D loss: 0.220750, acc.: 90.00%] [G loss: 6.305362]
9821 [D loss: 0.155089, acc.: 95.00%] [G loss: 6.069683]
9822 [D loss: 0.120461, acc.: 95.00%] [G loss: 5.697415]
9823 [D loss: 0.302172, acc.: 86.25%] [G loss: 4.800317]
9824 [D loss: 0.135087, acc.: 95.00%] [G loss: 5.070123]
9825 [D loss: 0.122804, acc.: 95.00%] [G loss: 4.985416]
9826 [D loss: 0.137840, acc.: 93.75%] [G loss: 4.129024]
9827 [D loss: 0.264756, acc.: 88.75%] [G loss: 7.898718]
9828 [D loss: 0.724631, acc.: 73.75%] [G loss: 8.721219]
9829 [D loss: 0.351976, acc.: 86.25%] [G loss: 4.506692]
9830 [D loss: 0.081453, acc.: 98.75%] [G loss: 5.252507]
9831 [D loss: 0.291094, acc.: 85.00%] [G loss: 4.704480]
9832 [D loss: 0.165118, acc.: 9

9959 [D loss: 0.146935, acc.: 93.75%] [G loss: 6.127773]
9960 [D loss: 0.427298, acc.: 81.25%] [G loss: 8.397059]
9961 [D loss: 0.428149, acc.: 80.00%] [G loss: 7.231782]
9962 [D loss: 0.112043, acc.: 96.25%] [G loss: 5.831824]
9963 [D loss: 0.140225, acc.: 95.00%] [G loss: 5.855962]
9964 [D loss: 0.180018, acc.: 91.25%] [G loss: 6.860184]
9965 [D loss: 0.512110, acc.: 76.25%] [G loss: 3.353870]
9966 [D loss: 0.093483, acc.: 96.25%] [G loss: 6.716902]
9967 [D loss: 0.207565, acc.: 91.25%] [G loss: 6.752100]
9968 [D loss: 0.214347, acc.: 92.50%] [G loss: 5.524586]
9969 [D loss: 0.153050, acc.: 90.00%] [G loss: 5.023531]
9970 [D loss: 0.183376, acc.: 91.25%] [G loss: 6.485265]
9971 [D loss: 0.297899, acc.: 88.75%] [G loss: 4.861373]
9972 [D loss: 0.055972, acc.: 97.50%] [G loss: 5.060346]
9973 [D loss: 0.324643, acc.: 82.50%] [G loss: 6.170869]
9974 [D loss: 0.218506, acc.: 90.00%] [G loss: 4.107806]
9975 [D loss: 0.089421, acc.: 98.75%] [G loss: 6.007463]
9976 [D loss: 0.379068, acc.: 8